In [1]:
import sys
import os
from pathlib import Path

import pandas as pd

# Add the sibling directory to Python's search path
sibling_dir = os.path.abspath("../utils")
sys.path.append(sibling_dir)

from db_utils import (
    get_securities_db,
    write_securities_db,
    get_holdings_db,
    write_holdings_db,
    get_id_adjustments_db,
    write_id_adjustments_db,
)

from openfigi_utils import (
    read_api_key,
    query_openfigi_by_sedol,
    figi_to_security,
)

#### Get a list of trusts with csv files 

In [4]:
root_directory = Path('./')

trusts = []
missing = []

for subdir in root_directory.iterdir():
    if subdir.is_dir():
        file_path = subdir / f'{subdir}.csv'
        if file_path.exists() and file_path.is_file():
            trusts.append(str(subdir))
        else:
            missing.append(str(subdir))

display(f'trusts: {trusts}')

if len(missing) > 0:
    display(f'missing: {missing}')

"trusts: ['brwm', 'jema', 'brsc', 'brge', 'brla', 'bemo']"

#### Test the first trust

In [3]:
trust = trusts[0]

df = pd.read_csv(root_directory / trust / f'{trust}.csv')
df

,SEDOL,Name,Holding,Value,Weight
0,B4T3BW6,Glencore,21852325.0,98575838.0,0.082
1,6144690,BHP,3211088.0,72405556.0,0.060
2,9315284,BHP Brazil Royalty,12000000.0,19935132.0,0.017
3,0144403,BHP,58636.0,1326932.0,0.001
4,0718875,Rio Tinto,1442689.0,75019828.0,0.062
...,...,...,...,...,...
69,7947221,Australian Carbon,10700000.0,57.0,0.000
70,9360225,Victorian Hydrogen & Ammonia Industry,7490000.0,40.0,0.000
71,7936955,Latrobe Fertilisers,10700000.0,0.0,0.000
72,B57R0L9,Polyus,111912.0,10.0,0.000


#### Get the dbs

In [4]:
securities_db = get_securities_db()
holdings_db = get_holdings_db()
id_adjustments_db = get_id_adjustments_db()
display(securities_db)
display(holdings_db)
display(id_adjustments_db)

,sedol,isin,figi,name,ticker,exchange,type,sector
0,6545464,,BBG000DKRVC9,ABU DHABI COMMERCIAL BANK,ADCB,UH,Common Stock,Equity
1,BN12D39,,BBG00M6JJSK9,ADNOC DRILLING CO PJSC,ADNOCDRI,UH,Common Stock,Equity
2,B03MN70,,BBG000BV9XY9,AKBANK T.A.S.,AKBNK,TI,Common Stock,Equity
3,B403QG4,,BBG000M294S2,MOUWASAT MEDICAL SERVICES CO,MOUWASAT,AB,Common Stock,Equity
4,B12LZH9,,BBG000FBK1L0,AL RAJHI BANK,RJHI,AB,Common Stock,Equity
...,...,...,...,...,...,...,...,...
59,B128FN6,,BBG000QS6Y35,CO FOR COOPERATIVE INSURANCE,TAWUNIYA,AB,Common Stock,Equity
60,BSHYYN1,,BBG0069P8N84,THE SAUDI NATIONAL BANK,SNB,AB,Common Stock,Equity
61,B03MYN3,,BBG000F101G0,TURKCELL ILETISIM HIZMET AS,TCELL,TI,Common Stock,Equity
62,BNGCVY1,,BBG008F2T3T2,UNITED CO RUSAL INTERNATIONA,RUAL,RM,Common Stock,Equity


,sedol,quantity,createdAt,updatedAt,deletedAt


,type,trust,id_in,id_out,createdAt,updatedAt,deletedAt
0,sedol,brwm,9346193,2857334,,,


In [14]:
# for _, row in df.iterrows():
#     id_type = 'sedol'
#     trust='brwm'
#     sedol = row['SEDOL']
#     assert isinstance(sedol, str), "sedol is not a string"
#     filtered_row = id_adjustments_db[(id_adjustments_db['type'] == 'sedol') & (id_adjustments_db['trust'] == 'brwm') & (str(id_adjustments_db['id_in']) == sedol)]

#     if not filtered_row.empty:
#         display(f'Adjustments for {id_type} {sedol} found')
#         display(filtered_row)

# filtered_row = id_adjustments_db[(id_adjustments_db['type'] == 'sedol') & (id_adjustments_db['trust'] == 'brwm') & (id_adjustments_db['id_in'] == '9346193')]
# filtered_row

id_adjustments_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   type       1 non-null      object
 1   trust      1 non-null      object
 2   id_in      1 non-null      int64 
 3   id_out     1 non-null      int64 
 4   createdAt  1 non-null      object
 5   updatedAt  1 non-null      object
 6   deletedAt  1 non-null      object
dtypes: int64(2), object(5)
memory usage: 188.0+ bytes


In [8]:
api_key = read_api_key('/home/craigc/.keys/openfigi')

failed = []

# for each SEDOL, check the securities db
for _, row in df.iterrows():
    sedol = row['SEDOL']
    name = row['Name']

    if not sedol in securities_db['sedol'].values:
        display(f'{sedol} ({name}) not in securities_db - adding')
        data = query_openfigi_by_sedol(api_key, sedol)
        if data:
            # display(data)
            security = figi_to_security(data, sedol)
            security_df = pd.DataFrame([security])
            securities_db = pd.concat([securities_db, security_df], ignore_index=True)
            display(f'{sedol} added to securities_db')
            break
        else:
            display(f'No data for {sedol} ({name})')
            failed.append((sedol, name))

securities_db

'6144690 (BHP) not in securities_db - adding'

'6144690 added to securities_db'

,sedol,isin,figi,name,ticker,exchange,type,sector
0,6545464,,BBG000DKRVC9,ABU DHABI COMMERCIAL BANK,ADCB,UH,Common Stock,Equity
1,BN12D39,,BBG00M6JJSK9,ADNOC DRILLING CO PJSC,ADNOCDRI,UH,Common Stock,Equity
2,B03MN70,,BBG000BV9XY9,AKBANK T.A.S.,AKBNK,TI,Common Stock,Equity
3,B403QG4,,BBG000M294S2,MOUWASAT MEDICAL SERVICES CO,MOUWASAT,AB,Common Stock,Equity
4,B12LZH9,,BBG000FBK1L0,AL RAJHI BANK,RJHI,AB,Common Stock,Equity
...,...,...,...,...,...,...,...,...
61,B03MYN3,,BBG000F101G0,TURKCELL ILETISIM HIZMET AS,TCELL,TI,Common Stock,Equity
62,BNGCVY1,,BBG008F2T3T2,UNITED CO RUSAL INTERNATIONA,RUAL,RM,Common Stock,Equity
63,B03MZJ6,,BBG000BVFT99,YAPI VE KREDI BANKASI,YKBNK,TI,Common Stock,Equity
64,B4T3BW6,NaN,BBG001MM1KV4,GLENCORE PLC,GLEN,LN,Common Stock,Equity


In [6]:
failed

[]

In [7]:
for _, row in df.iterrows():
    sedol = row['SEDOL']
    name = row['Name']

    display(sedol, name)
    break

'B4T3BW6'

'Glencore'